In [1]:
import nltk
from nltk.corpus import names
import random
from nltk.classify import NaiveBayesClassifier
from nltk.classify.util import accuracy
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import KFold
import numpy as np


nltk.download("names")




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/lailson/Documents/projetos/UFPI_PLN/env/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/lailson/Documents/projetos/UFPI_PLN/env/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/lailson/Documents/projetos/UFPI_PLN/env/lib/python3.12/site-packages/ipykernel/kernel

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



[nltk_data] Downloading package names to /Users/lailson/nltk_data...
[nltk_data]   Package names is already up-to-date!


True

In [5]:
#Mostar 10 primeiros nomes maculinos e femininos
nomes_masculinos = names.words("male.txt")
nomes_femininos = names.words("female.txt")

print("Primeiros 10 nomes masculinos:", nomes_masculinos[:10])
print("Primeiros 10 nomes femininos:", nomes_femininos[:10])

Primeiros 10 nomes masculinos: ['Aamir', 'Aaron', 'Abbey', 'Abbie', 'Abbot', 'Abbott', 'Abby', 'Abdel', 'Abdul', 'Abdulkarim']
Primeiros 10 nomes femininos: ['Abagael', 'Abagail', 'Abbe', 'Abbey', 'Abbi', 'Abbie', 'Abby', 'Abigael', 'Abigail', 'Abigale']


In [6]:
#Mostrar tamanho do corpus
print("Total de nomes masculinos:", len(nomes_masculinos))
print("Total de nomes femininos:", len(nomes_femininos))


Total de nomes masculinos: 2943
Total de nomes femininos: 5001


In [9]:
# Mostrar nomes aleatórios
nomes_rotulados = ([(nome, 'masculino') for nome in nomes_masculinos] +
                   [(nome, 'feminino') for nome in nomes_femininos])


import random
random.shuffle(nomes_rotulados)
print("Primeiros 10 nomes rotulados:", nomes_rotulados[:10])


Primeiros 10 nomes rotulados: [('Layton', 'masculino'), ('Demeter', 'feminino'), ('Willette', 'feminino'), ('Vergil', 'masculino'), ('Fiann', 'feminino'), ('Suzann', 'feminino'), ('Guido', 'masculino'), ('Rhianna', 'feminino'), ('Barry', 'feminino'), ('Titus', 'masculino')]


In [14]:

# Função para extrair features
def extrair_features(nome):
    features = {
        'ultima_letra': nome[-1].lower(),
        'primeira_letra': nome[0].lower(),
        'tamanho_nome': len(nome),
        'duas_ultimas_letras': nome[-2:].lower(),
        'segunda_letra': nome[1].lower() if len(nome) > 1 else '',                 
        'duas_primeiras_letras': nome[:2].lower(),                                 
        'numero_de_vogais': sum(1 for c in nome if c.lower() in 'aeiou'),           
        'numero_de_consoantes': sum(1 for c in nome if c.lower() not in 'aeiou'),   
        'quantidade_letras_repetidas': sum(nome.lower().count(c) > 1 for c in set(nome.lower())),
        'termina_com_vogal': nome[-1].lower() in 'aeiou',                          
        'quantidade_letras_unicas': len(set(nome.lower())),                        
    }
    return features


conjunto_features = [(extrair_features(nome), genero) for (nome, genero) in nomes_rotulados]
conjunto_features[0]


({'ultima_letra': 'n',
  'primeira_letra': 'l',
  'tamanho_nome': 6,
  'duas_ultimas_letras': 'on',
  'segunda_letra': 'a',
  'duas_primeiras_letras': 'la',
  'numero_de_vogais': 2,
  'numero_de_consoantes': 4,
  'quantidade_letras_repetidas': 0,
  'termina_com_vogal': False,
  'quantidade_letras_unicas': 6},
 'masculino')

In [23]:

# Proporcões
proporcoes = [0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95] 
resultados = []

for proporcao in proporcoes:
    tamanho_treino = int(proporcao * len(conjunto_features))
    conjunto_treino, conjunto_teste = conjunto_features[:tamanho_treino], conjunto_features[tamanho_treino:]
    
    classificador = NaiveBayesClassifier.train(conjunto_treino)
    
    y_true = [genero for (features, genero) in conjunto_teste]
    y_pred = [classificador.classify(features) for (features, genero) in conjunto_teste]
    
    acuracia = accuracy(classificador, conjunto_teste)
    
    precisao = precision_score(y_true, y_pred, pos_label='feminino')
    recall = recall_score(y_true, y_pred, pos_label='feminino')
    f1 = f1_score(y_true, y_pred, pos_label='feminino')
    
    resultados.append((proporcao, acuracia, precisao, recall, f1))
    
    print(f"Proporção de Treino/Teste: {int(proporcao * 100)}/{int((1 - proporcao) * 100)}%")
    print(f"Acurácia: {acuracia:.4f}, Precisão: {precisao:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}\n")

Proporção de Treino/Teste: 50/50%
Acurácia: 0.7784, Precisão: 0.8579, Recall: 0.7761, F1: 0.8150

Proporção de Treino/Teste: 60/40%
Acurácia: 0.7810, Precisão: 0.8561, Recall: 0.7801, F1: 0.8164

Proporção de Treino/Teste: 70/30%
Acurácia: 0.7794, Precisão: 0.8581, Recall: 0.7737, F1: 0.8137

Proporção de Treino/Teste: 75/25%
Acurácia: 0.7835, Precisão: 0.8620, Recall: 0.7771, F1: 0.8173

Proporção de Treino/Teste: 80/19%
Acurácia: 0.7879, Precisão: 0.8590, Recall: 0.7928, F1: 0.8246

Proporção de Treino/Teste: 85/15%
Acurácia: 0.7886, Precisão: 0.8611, Recall: 0.7923, F1: 0.8252

Proporção de Treino/Teste: 90/9%
Acurácia: 0.7962, Precisão: 0.8687, Recall: 0.7956, F1: 0.8305

Proporção de Treino/Teste: 95/5%
Acurácia: 0.7990, Precisão: 0.8717, Recall: 0.7944, F1: 0.8312



Com a propoção de 95% para treino e 5% para teste tivemos a melhor acurácia de 0.7990.
Vamos aplicar a validação cruzada para verificar se a proporção se mantem.

In [26]:
# Avaliação para cada proporção usando K-Fold Cross-Validation
for proporcao in proporcoes:
    print(f"\nProporção de Treino/Teste: {int(proporcao * 100)}/{int((1 - proporcao) * 100)}%")
    
    tamanho_treino = int(proporcao * len(conjunto_features))
    conjunto_treino = conjunto_features[:tamanho_treino]
    conjunto_teste = conjunto_features[tamanho_treino:]
    kf = KFold(n_splits=10, shuffle=True, random_state=42)  

    
    acuracias = []
    precisoes = []
    recalls = []
    f1_scores = []

    for treino_index, teste_index in kf.split(conjunto_treino):
        fold_treino = [conjunto_treino[i] for i in treino_index]
        fold_teste = [conjunto_treino[i] for i in teste_index]
        classificador = NaiveBayesClassifier.train(fold_treino)

        y_true = [genero for (_, genero) in fold_teste]
        y_pred = [classificador.classify(features) for (features, _) in fold_teste]

        acuracia = accuracy(classificador, fold_teste)
        precisao = precision_score(y_true, y_pred, pos_label='feminino')
        recall = recall_score(y_true, y_pred, pos_label='feminino')
        f1 = f1_score(y_true, y_pred, pos_label='feminino')

        acuracias.append(acuracia)
        precisoes.append(precisao)
        recalls.append(recall)
        f1_scores.append(f1)

    print(f"Acurácia Média: {np.mean(acuracias):.4f}")
    print(f"Precisão Média: {np.mean(precisoes):.4f}")
    print(f"Recall Médio: {np.mean(recalls):.4f}")
    print(f"F1-Score Médio: {np.mean(f1_scores):.4f}")



Proporção de Treino/Teste: 50/50%
Acurácia Média: 0.7772
Precisão Média: 0.8567
Recall Médio: 0.7761
F1-Score Médio: 0.8141

Proporção de Treino/Teste: 60/40%
Acurácia Média: 0.7761
Precisão Média: 0.8610
Recall Médio: 0.7711
F1-Score Médio: 0.8132

Proporção de Treino/Teste: 70/30%
Acurácia Média: 0.7793
Precisão Média: 0.8629
Recall Médio: 0.7745
F1-Score Médio: 0.8160

Proporção de Treino/Teste: 75/25%
Acurácia Média: 0.7764
Precisão Média: 0.8628
Recall Médio: 0.7682
F1-Score Médio: 0.8124

Proporção de Treino/Teste: 80/19%
Acurácia Média: 0.7800
Precisão Média: 0.8638
Recall Médio: 0.7722
F1-Score Médio: 0.8152

Proporção de Treino/Teste: 85/15%
Acurácia Média: 0.7771
Precisão Média: 0.8647
Recall Médio: 0.7659
F1-Score Médio: 0.8121

Proporção de Treino/Teste: 90/9%
Acurácia Média: 0.7769
Precisão Média: 0.8618
Recall Médio: 0.7688
F1-Score Médio: 0.8125

Proporção de Treino/Teste: 95/5%
Acurácia Média: 0.7784
Precisão Média: 0.8636
Recall Médio: 0.7695
F1-Score Médio: 0.8136


A proporção 70/30 apresentou o melhor acurácia média 0.7793 e também oo melhor f1-score de 0.8160

Sem validação cruzada: 95/5
Com validação cruzada: 70/30

In [18]:
# Mostrar as features mais informativas
print("\nFeatures mais informativas:")
classificador.show_most_informative_features(20)


Features mais informativas:
Most Informative Features
     duas_ultimas_letras = 'na'           femini : mascul =     96.1 : 1.0
     duas_ultimas_letras = 'ia'           femini : mascul =     89.8 : 1.0
     duas_ultimas_letras = 'la'           femini : mascul =     71.4 : 1.0
            ultima_letra = 'a'            femini : mascul =     36.8 : 1.0
     duas_ultimas_letras = 'sa'           femini : mascul =     31.4 : 1.0
     duas_ultimas_letras = 'ta'           femini : mascul =     31.1 : 1.0
            ultima_letra = 'k'            mascul : femini =     29.8 : 1.0
     duas_ultimas_letras = 'us'           mascul : femini =     27.4 : 1.0
     duas_ultimas_letras = 'ra'           femini : mascul =     25.1 : 1.0
     duas_ultimas_letras = 'ch'           mascul : femini =     25.0 : 1.0
     duas_ultimas_letras = 'do'           mascul : femini =     25.0 : 1.0
     duas_ultimas_letras = 'ld'           mascul : femini =     24.3 : 1.0
     duas_ultimas_letras = 'rd'           mas

In [27]:
nome_novo = "Lailson"
features_nome_novo = extrair_features(nome_novo)

genero_previsto = classificador.classify(features_nome_novo)
print(f"O gênero previsto para o nome '{nome_novo}' é: {genero_previsto}")


O gênero previsto para o nome 'Lailson' é: masculino


In [28]:
nome_novo2 = "Henrique"
features_nome_novo2 = extrair_features(nome_novo2)

genero_previsto = classificador.classify(features_nome_novo2)
print(f"O gênero previsto para o nome '{nome_novo2}' é: {genero_previsto}")

O gênero previsto para o nome 'Henrique' é: feminino
